In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [4]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.input = nn.Linear(in_features=4, out_features=16)
        self.hidden_1 = nn.Linear(in_features=16, out_features=16)
        self.output = nn.Linear(in_features=16, out_features=3)
        
    def forward(self, x):
        x = F.relu(self.input(x))
        x = F.relu(self.hidden_1(x))
    
model = Net()

In [ ]:
X = torch.rand(3, 4)
Y = torch.rand(1, 3)
print(X.shape)
print(Y.shape)

In [ ]:
from torchviz import make_dot

model = Net()
y = model(X)

make_dot(y.mean(), params=dict(model.named_parameters()))

In [ ]:
import os
import torch
import torch.nn as nn
from datasets import load_dataset
from transformers import get_scheduler

##### Own
import train_utils.cifar_utils as cifar_utils
from train_utils import make_optimizer, get_cfg

from vision_transformer import VisionTransformer
from train_utils import cifar_utils

In [12]:
cfg = get_cfg("config/vit_train.yml")
cfg["training"]["batch_size"] = 1
####### Dataset setup #######
dataset_cfg = cfg["cifar_dataset"]

label2id, id2label = cifar_utils.get_label_dicts(dataset_cfg["label_type"])

train_dataloader, validation_dataloader, test_dataloader = cifar_utils.dataloaders_from_cfg(cfg)

model = VisionTransformer(
    image_size=cfg["cifar_dataset"]["image_size"], use_linear_patch=True, num_classes=len(label2id.keys()))

In [ ]:
# Passing single batch to model
X = next(iter(train_dataloader))
y = model(X["pixel_values"])

print(X["pixel_values"].shape)
print(y.shape)

In [ ]:
# torchviz visualization
make_dot(y.mean(), params=dict(model.named_parameters()))

In [22]:
# onnx
torch.onnx.export(model=model,args=X["pixel_values"], f="model.onnx", input_names=["input"], output_names=["output"])